<a href="https://colab.research.google.com/github/simonmoedinger/llm_examples/blob/main/Llama_2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama-2-7b-chat-hf

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
!pip install pypdf

# Hugging Face Login

https://huggingface.co/docs/huggingface_hub/quick-start

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

# Import Libraries

In [5]:
from langchain.llms import HuggingFacePipeline



In [6]:
from transformers import AutoTokenizer

In [7]:
import transformers

In [8]:
import torch

In [9]:
import warnings
warnings.filterwarnings('ignore')


In [12]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import torch
import pandas as pd
import math
from sklearn.metrics import accuracy_score

## Load Llama-2-7b-chat-hf

In [13]:
model_name="meta-llama/Llama-2-7b-chat-hf"

In [14]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id=tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
generation_config=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

In [17]:
llm=HuggingFacePipeline(pipeline=generation_config, model_kwargs={'temperature':0})

In [28]:
prompt="Was ist ein Auto?"

In [29]:
print(llm(prompt))

OutOfMemoryError: ignored

In [20]:
def prompt_model(prompt):
    input_tokens=tokenizer(prompt + " Die richtige Antwort ist BEGINANTWORT", return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [21]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."
prompt_no_case = "Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."

In [22]:
questions_df = pd.read_csv("/output.csv")
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [23]:
questions_df

,Case,Question,Answer 1,Answer 2,Answer 3,Answer 4,Answer 5,Correct Answer,Answer LLM
0,Eine 35-jährige Patientin wird in Begleitung d...,Welche der genannten Vorgehensweisen ist bei d...,0) Anlage einer Hirndrucksonde,1) Polytrauma-CT-Spirale,2) MRT des Abdomens,3) Anlage einer Thoraxdrainage links,4) 12-Kanal-EKG,1,-1
1,Eine 35-jährige Patientin wird in Begleitung d...,Bei weiterhin kritisch-instabiler Patientin se...,"0) abwartendes, konservatives Vorgehen",1) sofortige Notfalloperation,2) Punktion des Abdomens mit Anlage einer Drai...,3) Anlage einer Thoraxdrainage links,4) Angio-CT des Schädels,1,-1
2,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten diagnostisch-pathogenetis...,0) eine durch eine Schädigung der Pyramidenbah...,1) eine infolge einer Schädigung der linken Py...,2) eine auf einer Schädigung des Kleinhirns be...,3) ein durch eine Meningitis bedingtes Lhermit...,4) zentral bedingte neurologische Zeichen und ...,4,-1
3,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten Therapieoptionen einschli...,"0) Entfernung der Bandscheibe, um eine weitere...",1) Verzicht auf eine Operation wegen schon ein...,2) Radiochemotherapie zur Entfernung des Wirbe...,3) Wirbelkörperersatz zur Entfernung der degen...,4) Verordnung neurotroper Medikamente zum Wied...,0,-1
4,Bei der Früherkennungsuntersuchung auf Gebärmu...,Welcher der folgenden Virustypen gehört hinsic...,0) 1,1) 4,2) 11,3) 16,4) 20,3,-1
...,...,...,...,...,...,...,...,...,...
243,NaN,Ein 32-jähriger Mann stellt sich bei einem HNO...,0) Otosklerose,1) Perilymphfistel des lateralen Bogengangs,2) Fehlbildung der Cochlea,3) traumatische Trommelfellperforation,4) Dehiszenz des superioren Bogengangs,4,-1
244,NaN,"Klaus B., 26-jähriger Sportstudent, lebt seit ...",0) einmalige Tetanus-Diphtherie-Pertussis-Impf...,1) einmalige Masern-Mumps-Röteln-Impfung und P...,2) einmalige Tetanus-Diphtherie-Pertussis-Impf...,3) einmalige Impfung gegen Hepatitis A und B,4) einmalige Pneumokokken-Impfung und einmalig...,1,-1
245,NaN,Die Wirksamkeit eines Impfstoffes oder VE („va...,0) 20 %,1) 40 %,2) 60 %,3) 80 %,4) 100 %,3,-1
246,NaN,Ein 46-jähriger Patient ohne relevante bekannt...,0) Alpha-Motoneuron im Rückenmark-Vorderhorn,1) Pyramidenbahn,2) Purkinje-Zellen,3) Fasciculus gracilis,4) dorsolaterales Putamen,1,-1


In [25]:
torch.cuda.empty_cache()

In [26]:
for index, row in questions_df.iterrows():
    if row["Answer LLM"] == -1:
        if type(row["Case"]) == str:
            prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
        else:
            prompt = prompt_no_case
        prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
        for i in range(5):
            prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)].replace(str(i) + ")",""))
        answer = prompt_model(prompt).split("Die richtige Antwort ist BEGINANTWORT")[1].split("ENDANTWORT")[0]
        found = False
        for i in range(5):
            if row["Answer "+ str(i+1)].replace(str(i) + ")","").strip().lower() == answer.strip().lower():
                found = i
                break
        #print(answer)
        if found != False:
            questions_df.loc[index, "Answer LLM"] = found
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))



OutOfMemoryError: ignored

In [ ]:
questions_df.to_csv("llama_output.csv")

In [ ]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [ ]:
print("Accuracy:",accuracy_score(y_true, y_pred))

##**Step 05: Prompt Templates**

Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In many Large Language Model applications we donot pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template

#Import All the Required Libraries

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.chains import LLMChain

#**Example 1**

In [ ]:
prompt_template=PromptTemplate(input_variables=["cuisine"],
                               template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this")

In [ ]:
input_prompt=prompt_template.format(cuisine="Italian")

In [ ]:
print(input_prompt)

I want to open a restaurant for Italian food. Suggest a fency name for this


##**Step 06: Chains**

Combine LLMs and Prompts in multi-step workflows
- The simplest and most common type of Chain is LLMChain, which passes the input first to Prompt Template and then to the Large Language Model
- LLMChain is responsible to execute the PromptTemplate, For every PromptTemplate we will specifically have an LLMChain

#**Example 1**

In [ ]:
prompt_template=PromptTemplate(input_variables=["book_name"],
                               template="Proivde me a concise summary of the book {book_name}")

Whatever input text i am giving that will get assigned to this particular variable that is **book name**

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)
response= chain.run("Alchemist")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Proivide me a concise summary of the book Alchemist

> Finished chain.
 by Paulo Coelho.

The Alchemist is a novel by Paulo Coelho, first published in 1988. The story follows a young shepherd named Santiago, who is on a journey to fulfill his personal legend and find his treasure. The book is an allegory, with themes of spirituality, self-discovery, and the power of belief.

Santiago's journey takes him from his home in Spain to Egypt, where he meets various characters who teach him valuable life lessons. Along the way, he learns about the importance of listening to his heart and following his dreams, even when they go against the expectations of others.

Throughout the book, Santiago encounters various alchemical symbols, such as the alchemist's stone, which represents the power of belief and the ability to turn lead into gold. He also learns about the importance of patience, perseverance, and the interconnectedness of all th